To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

Features in the notebook:
1. Uses Maxime Labonne's [FineTome 100K](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset.
1. Convert ShareGPT to HuggingFace format via `standardize_sharegpt`
2. Train on Completions / Assistant only via `train_on_responses_only`
3. Unsloth now supports Torch 2.4, all TRL & Xformers versions & Python 3.12!

In [1]:
# %%capture
!pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.5/203.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
from tqdm import tqdm
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [5]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [6]:
import json
# Path to the JSON file
json_file_path = '/content/debug_game_dataset.json'

# Read the JSON file and load the data
with open(json_file_path, 'r') as json_file:
    new_conv = json.load(json_file)

In [7]:
old_conv = dataset["conversations"][0:50000]
len(old_conv)

50000

In [8]:
full_conv = new_conv#+old_conv
len(full_conv)

63

In [9]:
from datasets import Dataset
data_dict = {"conversations": full_conv}
dataset = Dataset.from_dict(data_dict)

In [10]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 63
})

We look at how the conversations are structured for item 5:

In [11]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

In [14]:
dataset[62]["conversations"]

[{'content': 'You are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your questions should be clear, engaging, and age-appropriate.',
  'role': 'system'},
 {'content': 'Generate a Python debugging challenge for kids about loops at medium level.',
  'role': 'user'},
 {'content': "<ORIGINAL_CODE>\ntotal = 0\nfor i in range(1, 11):\n    total += i\nprint('Sum:', total)\n</ORIGINAL_CODE>\n\n<BUGGY_CODE>\ntotal = 0\nfor i in range(1, 10):\n    total += i\nprint('Sum:', total)\n</BUGGY_CODE>\n\n<QUESTION>\nThis code should calculate the sum of numbers 1 to 10, but the result is 45 instead of 55. Why?\n</QUESTION>\n\n<OPTIONS>\n1. total should start at 1, not 0\n2. The range should be (1, 11) not (1, 10)\n3. There's a calculation error in the addition\n4. The loop should start at 0, not 1\n</OPTIONS>\n\n<ANSWER>\nAnswer: 2\n</ANSWER>\n\n<EXPLANATION>\nThe range(1, 10) includes numbers from 1 to 9, not including 10. To include 10, it 

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [15]:
dataset[62]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your questions should be clear, engaging, and age-appropriate.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate a Python debugging challenge for kids about loops at medium level.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<ORIGINAL_CODE>\ntotal = 0\nfor i in range(1, 11):\n    total += i\nprint('Sum:', total)\n</ORIGINAL_CODE>\n\n<BUGGY_CODE>\ntotal = 0\nfor i in range(1, 10):\n    total += i\nprint('Sum:', total)\n</BUGGY_CODE>\n\n<QUESTION>\nThis code should calculate the sum of numbers 1 to 10, but the result is 45 instead of 55. Why?\n</QUESTION>\n\n<OPTIONS>\n1. total should start at 1, not 0\n2. The range should be (1, 11) not (1, 10)\n3. There's a calculation error in the addition\n4. The loop should start at 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/63 [00:00<?, ? examples/s]

In [ ]:
'''
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from datasets import Dataset
import random
import numpy as np
import torch

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(3407)

# Improved training arguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,  # Increased from 4
        warmup_ratio=0.1,  # Using ratio instead of steps
        num_train_epochs=2,  # Doubled from 1
        learning_rate=1e-5,  # Reduced from 2e-4 for finer tuning
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Changed from linear to cosine
        seed=3407,
        output_dir="outputs",
        report_to="none",
        save_strategy="epoch",  # Save at the end of each epoch
        #evaluation_strategy="steps" if eval_dataset else "no",  # Add if you have an eval dataset
        #eval_steps=100 if eval_dataset else None,  # Add if you have an eval dataset
    ),
)
'''

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [17]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/63 [00:00<?, ? examples/s]

We verify masking is actually done:

In [18]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your questions should be clear, engaging, and age-appropriate.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate a Python debugging challenge for kids about functions at hard level.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<ORIGINAL_CODE>\ndef factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n - 1)\n\nprint(factorial(5))\n</ORIGINAL_CODE>\n\n<BUGGY_CODE>\ndef factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n)\n\nprint(factorial(5))\n</BUGGY_CODE>\n\n<QUESTION>\nThis recursive factorial function causes a maximum recursion depth error. Why?\n</QUESTION>\n\n<OPTIONS>\n1. The base case condition is incor

In [19]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                    <ORIGINAL_CODE>\ndef factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n - 1)\n\nprint(factorial(5))\n</ORIGINAL_CODE>\n\n<BUGGY_CODE>\ndef factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n)\n\nprint(factorial(5))\n</BUGGY_CODE>\n\n<QUESTION>\nThis recursive factorial function causes a maximum recursion depth error. Why?\n</QUESTION>\n\n<OPTIONS>\n1. The base case condition is incorrect\n2. The recursive call doesn't decrement n, causing infinite recursion\n3. Factorial should be calculated using a loop, not recursion\n4. The function should return n * (n-1) directly\n</OPTIONS>\n\n<ANSWER>\nAnswer: 2\n</ANSWER>\n\n<EXPLANATION>\nThe recursive call is factorial(n) instead of factorial(n - 1), which means n never decreases, causing infinite recursion. Each call should reduce n by 1 to eventually reach the

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 63 | Num Epochs = 13 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712/3,000,000,000 (1.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.307300
2,1.376000
3,1.181300
4,1.176800
5,1.098100
6,1.028700
7,0.965100
8,0.932900
9,0.718800
10,0.675000


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [22]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "system",
        "content": "You are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your debugging puzzles should be clear, engaging, and age-appropriate. Each challenge should include original correct code, buggy code with intentional errors, a question about what's wrong, multiple-choice options to fix the bug, the correct answer, a helpful explanation, difficulty level, and programming topic category."
    },
    {
        "role": "user",
        "content": "Generate a Python debugging challenge for kids about loops at medium level."
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 0.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your debugging puzzles should be clear, engaging, and age-appropriate. Each challenge should include original correct code, buggy code with intentional errors, a question about what's wrong, multiple-choice options to fix the bug, the correct answer, a helpful explanation, difficulty level, and programming topic category.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate a Python debugging challenge for kids about loops at medium level.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<ORIGINAL_CODE>\ntotal = 0\nfor i in range(1, 5):\n    total += i\nprint('Sum:', total)\n</ORIGINAL_CODE>\n\n<BUGGY_CODE>\ntotal = 0\nfor i in range(1, 5):\n    total = i\nprint('Sum:', total"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [23]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "system",
        "content": "You are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your debugging puzzles should be clear, engaging, and age-appropriate. Each challenge should include original correct code, buggy code with intentional errors, a question about what's wrong, multiple-choice options to fix the bug, the correct answer, a helpful explanation, difficulty level, and programming topic category."
    },
    {
        "role": "user",
        "content": "Generate a Python debugging challenge for kids about loops at medium level."
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.5, min_p = 0.1)

<ORIGINAL_CODE>
total = 0
for i in range(1, 5):
    total += i
print('Sum:', total)
</ORIGINAL_CODE>

<BUGGY_CODE>
total = 0
for i in range(1, 5):
    total = i
print('Sum:', total)
</BUGGY_CODE>

<QUESTION>
This code should calculate the sum of numbers 1 to 4, but the result is 4 instead of 14. What's wrong?
</QUESTION>

<OPTIONS>
1. total should be initialized before the loop
2. Using assignment assignment = i instead of adding


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [24]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [25]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in Paris, France, is a recognizable landmark. Although it's not the tallest structure, it's an iconic and impressive tower. It stands 324 meters (1,063 feet) tall, including its antenna, and was built as the entrance arch for the 1889 World's Fair.

The Eiffel Tower has four main pillars supporting it, and its lattice-like structure is made up of interlocking girders. The tower is painted a distinctive brown color to protect it from rust, and it's anchored to the ground with deep foundations.

During the day, the tower is often shadowed


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [26]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [27]:
# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [28]:
# Save to 8bit Q8_0
#if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
#if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
#if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
#if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
#if False:
 #   model.push_to_hub_gguf(
  #      "hf/model", # Change hf to your username!
   #     tokenizer,
    #    quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
     #   token = "", # Get a token at https://huggingface.co/settings/tokens
    #)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.6 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 21.05it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf:

In [ ]:
# Debugging Game Model Testing Code
!pip install -q transformers torch numpy tqdm datasets accelerate bitsandbytes sentencepiece
!pip install axolotl
import os
import json
import torch
import random
import numpy as np
from tqdm import tqdm
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from axolotl.models import FastLanguageModel

# Install required packages

# Fix the random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

# 1. Model Loading
def load_model(model_path="lora_model"):
    """Loads the fine-tuned Llama model."""
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)  # Enable fast inference
    return model, tokenizer

# Fallback method if FastLanguageModel doesn't work
def load_model_fallback(model_path="lora_model"):
    """Fallback method to load the model using standard HuggingFace methods."""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        load_in_4bit=True
    )
    return model, tokenizer

# 2. Text Generation Function
def generate_response(model, tokenizer, prompt, max_length=2048, temperature=0.7):
    """Generate a response from the model."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate with specified parameters
    output = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract only the generated part, not including the prompt
    generated_text = response[len(tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)):]

    return generated_text.strip()

# 3. Test Data Loading
def load_test_data(file_path="/content/debug_game_dataset.json"):
    """Load test examples from the JSON dataset."""
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Convert to a format suitable for testing
    test_examples = []
    for conversation in data:
        # Extract system message
        system_msg = conversation[0]["content"]

        # Extract user query (category and difficulty)
        user_query = conversation[1]["content"]

        # Extract expected response
        expected_response = conversation[2]["content"]

        test_examples.append({
            "system_msg": system_msg,
            "user_query": user_query,
            "expected_response": expected_response
        })

    return test_examples

# 4. Build Prompts
def build_prompt(system_msg, user_query):
    """Build a properly formatted prompt for the model."""
    return f"""<s>[INST] <<SYS>>
{system_msg}
<</SYS>>

{user_query} [/INST]"""

# 5. Evaluation Helper Functions
def extract_sections(response):
    """Extract structured sections from the response."""
    sections = {}

    # Define the sections to extract
    section_markers = [
        ("ORIGINAL_CODE", "ORIGINAL_CODE", "/ORIGINAL_CODE"),
        ("BUGGY_CODE", "BUGGY_CODE", "/BUGGY_CODE"),
        ("QUESTION", "QUESTION", "/QUESTION"),
        ("OPTIONS", "OPTIONS", "/OPTIONS"),
        ("ANSWER", "ANSWER", "/ANSWER"),
        ("EXPLANATION", "EXPLANATION", "/EXPLANATION"),
        ("DIFFICULTY", "DIFFICULTY", "/DIFFICULTY"),
        ("CATEGORY", "CATEGORY", "/CATEGORY")
    ]

    for section_name, start_marker, end_marker in section_markers:
        start_idx = response.find(f"<{start_marker}>")
        if start_idx != -1:
            content_start = start_idx + len(f"<{start_marker}>")
            end_idx = response.find(f"</{end_marker}>", content_start)
            if end_idx != -1:
                content = response[content_start:end_idx].strip()
                sections[section_name] = content

    return sections

def evaluate_response(response):
    """Evaluate if a response contains all required sections."""
    sections = extract_sections(response)

    # Check if all required sections are present
    required_sections = ["BUGGY_CODE", "QUESTION", "OPTIONS", "ANSWER", "EXPLANATION"]
    sections_present = all(section in sections for section in required_sections)

    # Calculate completeness score (percentage of required sections present)
    completeness = sum(1 for section in required_sections if section in sections) / len(required_sections)

    return {
        "is_complete": sections_present,
        "completeness_score": completeness,
        "sections_present": list(sections.keys()),
        "missing_sections": [section for section in required_sections if section not in sections]
    }

# 6. Main Testing Function
def test_model(model_path="lora_model", test_data_path="debug_game_dataset.json", num_examples=5):
    """Test the model on a sample of debugging challenges."""
    print(f"Loading model from {model_path}...")
    try:
        model, tokenizer = load_model(model_path)
    except Exception as e:
        print(f"Error loading model with FastLanguageModel: {str(e)}")
        print("Falling back to standard loading method...")
        model, tokenizer = load_model_fallback(model_path)

    print("Loading test data...")
    test_examples = load_test_data(test_data_path)

    # Sample a subset for testing if needed
    if num_examples < len(test_examples):
        test_subset = random.sample(test_examples, num_examples)
    else:
        test_subset = test_examples

    print(f"Testing model on {len(test_subset)} examples...")

    results = []
    for i, example in enumerate(tqdm(test_subset)):
        print(f"\n\n===== Testing Example {i+1}/{len(test_subset)} =====")

        # Build the prompt
        prompt = build_prompt(example["system_msg"], example["user_query"])

        # Generate response
        try:
            response = generate_response(model, tokenizer, prompt)
            print(f"\nGenerated Response:\n{response[:500]}...")  # Print first 500 chars

            # Evaluate response
            evaluation = evaluate_response(response)
            results.append(evaluation)

            print(f"\nEvaluation:")
            print(f"- Complete: {evaluation['is_complete']}")
            print(f"- Completeness Score: {evaluation['completeness_score']:.2f}")
            print(f"- Sections Present: {', '.join(evaluation['sections_present'])}")
            if evaluation['missing_sections']:
                print(f"- Missing Sections: {', '.join(evaluation['missing_sections'])}")

        except Exception as e:
            print(f"Error during generation: {str(e)}")
            results.append({"error": str(e)})

    # Calculate overall statistics
    complete_responses = sum(1 for r in results if r.get('is_complete', False))
    avg_completeness = np.mean([r.get('completeness_score', 0) for r in results])

    print("\n===== Overall Results =====")
    print(f"Complete responses: {complete_responses}/{len(test_subset)} ({complete_responses/len(test_subset)*100:.2f}%)")
    print(f"Average completeness score: {avg_completeness:.2f}")

    return results

# 7. Testing Different Categories and Difficulties
def test_by_category_and_difficulty(model, tokenizer, categories, difficulties):
    """Test model on specific categories and difficulties."""
    system_msg = "You are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your questions should be clear, engaging, and age-appropriate."

    results = {}

    for category in categories:
        results[category] = {}
        for difficulty in difficulties:
            print(f"\n===== Testing {category} at {difficulty} level =====")

            user_query = f"Generate a Python debugging challenge for kids about {category.lower()} at {difficulty.lower()} level."
            prompt = build_prompt(system_msg, user_query)

            response = generate_response(model, tokenizer, prompt)
            evaluation = evaluate_response(response)

            print(f"\nGenerated Response (preview):\n{response[:300]}...")
            print(f"\nEvaluation:")
            print(f"- Complete: {evaluation['is_complete']}")
            print(f"- Completeness Score: {evaluation['completeness_score']:.2f}")

            results[category][difficulty] = {
                "response": response,
                "evaluation": evaluation
            }

    return results

# 8. Interactive Testing Function
def interactive_test(model, tokenizer):
    """Allow interactive testing of the model with custom prompts."""
    system_msg = "You are an AI tutor that creates debugging challenges to teach Python programming concepts to children aged 12-15. Your questions should be clear, engaging, and age-appropriate."

    print("\n===== Interactive Testing Mode =====")
    print("Type 'exit' to quit.")

    while True:
        user_input = input("\nEnter your prompt (e.g., 'Generate a Python debugging challenge about loops'): ")

        if user_input.lower() == 'exit':
            break

        prompt = build_prompt(system_msg, user_input)

        print("\nGenerating response...")
        response = generate_response(model, tokenizer, prompt)

        print("\n===== Generated Response =====")
        print(response)

        evaluation = evaluate_response(response)
        print("\n===== Evaluation =====")
        print(f"Complete: {evaluation['is_complete']}")
        print(f"Completeness Score: {evaluation['completeness_score']:.2f}")
        print(f"Sections Present: {', '.join(evaluation['sections_present'])}")
        if evaluation['missing_sections']:
            print(f"Missing Sections: {', '.join(evaluation['missing_sections'])}")

# 9. Save and Compare Results
def save_results(results, filename="model_test_results.json"):
    """Save test results to a JSON file."""
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"Results saved to {filename}")

# 10. Main Execution
if __name__ == "__main__":
    # Check for GPU
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")

    # Set the model path (update this to your actual model path)
    MODEL_PATH = "lora_model"  # or use a specific path

    # Choose testing mode
    print("\nChoose testing mode:")
    print("1. Run automated tests")
    print("2. Test specific categories and difficulties")
    print("3. Interactive testing")

    mode = input("Enter mode (1-3): ")

    try:
        # Load model (do this once for all modes)
        model, tokenizer = load_model(MODEL_PATH)

        if mode == "1":
            # Run automated tests
            num_examples = int(input("Number of examples to test (default 5): ") or "5")
            results = test_model(MODEL_PATH, "debug_game_dataset.json", num_examples)
            save_results(results)

        elif mode == "2":
            # Test specific categories and difficulties
            categories = [
                "Syntax Errors",
                "Conditional Logic",
                "Loops",
                "Functions",
                "Lists",
                "Dictionaries",
                "String Manipulation"
            ]
            difficulties = ["Easy", "Medium", "Hard"]

            print("\nAvailable categories:")
            for i, cat in enumerate(categories):
                print(f"{i+1}. {cat}")

            cat_indices = input("Enter category numbers to test (comma-separated, or 'all'): ")

            if cat_indices.lower() == 'all':
                selected_categories = categories
            else:
                selected_indices = [int(idx.strip())-1 for idx in cat_indices.split(",")]
                selected_categories = [categories[idx] for idx in selected_indices if 0 <= idx < len(categories)]

            print("\nSelected categories:", ", ".join(selected_categories))

            results = test_by_category_and_difficulty(model, tokenizer, selected_categories, difficulties)
            save_results(results, "category_test_results.json")

        elif mode == "3":
            # Interactive testing
            interactive_test(model, tokenizer)

        else:
            print("Invalid mode selected.")

    except Exception as e:
        print(f"Error: {str(e)}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>